In [1]:
import pandas as pd
import re
import yaml
import sqlparse
import os
import pandas as pd
import numpy as np
import requests
from IPython.display import display, Markdown

from langchain.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings, ChatOpenAI

In [2]:
def add_repo_root_path():
    import os
    import sys
    repo_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
    if repo_root not in sys.path:
        sys.path.append(repo_root)
        
add_repo_root_path()
from src import generate_knowledge
from src import create_rag_db
from src import llm_chain_tools
from src.enhanced_retriever import EnhancedRetriever

In [3]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 50)
#pd.set_option('display.width', None)
#pd.set_option('display.max_colwidth', 10) 

### INIT

In [4]:
generate_knowledge.add_repo_root_path()
import openai_setup

OPENAI_API_KEY = openai_setup.conf['key']
OPENAI_PROJECT = openai_setup.conf['project']
OPENAI_ORGANIZATION = openai_setup.conf['organization']
DEFAULT_LLM_MODEL = "gpt-4o-mini"
CHROMADB_DIRECTORY = '../chromadb'
COLLECTION_NAME = "my_chromadb" 

import os
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
os.environ['OPENAI_MODEL_NAME'] = DEFAULT_LLM_MODEL


In [5]:
langchain_openai_embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY, model="text-embedding-ada-002")
langchain_openai_llm = ChatOpenAI(model=DEFAULT_LLM_MODEL, temperature=0.1, openai_api_key=OPENAI_API_KEY, openai_organization = OPENAI_ORGANIZATION)

In [6]:
# Define file paths for YAML configurations
files = {
    'agents': '../config/agents.yml',
    'tasks': '../config/tasks.yml'
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
agents_config = configs['agents']
tasks_config = configs['tasks']

print(agents_config)
print(tasks_config)

{'interpretation_agent': {'role': 'Request Interpreter\n', 'goal': 'Interpret user requests related to dbt projects and translate them into actionable decisions. Use expertise in dbt, data modeling, and analytics engineering to determine the type of action required.\n', 'backstory': "You specialize in analyzing requests to identify whether the action involves adding a field, modifying an existing model, or retrieving specific information. Your goal is to provide concise and actionable outputs tailored to the user's needs.\n", 'verbose': True, 'allow_delegation': False}, 'evaluation_agent': {'role': 'Evaluation Specialist\n', 'goal': 'Evaluate user requests related to dbt projects and provide concise, actionable insights and steps required to address the request. Leverage expertise in data modeling, dbt project structure, and dependency analysis to ensure accurate evaluations.\n', 'backstory': 'You specialize in analyzing interpreted requests and breaking them down into specific, action

In [7]:
from crewai import Agent, Task, Crew

### TESTS

#### Agents

In [8]:

# Creating Agents
interpretation_agent = Agent(
  config=agents_config['interpretation_agent'],
)

evaluation_agent = Agent(
  config=agents_config['evaluation_agent'],
)

lineage_agent = Agent(
  config=agents_config['lineage_agent'],
)

plan_agent = Agent(
  config=agents_config['plan_agent'],
)

In [9]:
# Creating Tasks
interpretation_task = Task(
  config=tasks_config['interpretation_task'],
  agent=interpretation_agent
)

evaluation_task = Task(
  config=tasks_config['evaluation_task'],
  agent=evaluation_agent
)

lineage_task = Task(
  config=tasks_config['lineage_task'],
  agent=lineage_agent
)

plan_task = Task(
  config=tasks_config['plan_task'],
  agent=plan_agent
)

In [10]:
crew = Crew(
  agents=[
    interpretation_agent,
    evaluation_agent
  ],
  tasks=[
    interpretation_task,
    evaluation_task
  ],
  verbose=True
)

In [11]:
user_input = 'Give me all the information about the models related with customers'

inputs = {
  'request': user_input
}

# Run the crew
result = crew.kickoff(
  inputs=inputs
)

ValueError: Missing required template variable 'interpretation' in description

#### Flows

In [12]:
import nest_asyncio
nest_asyncio.apply()

In [12]:
from crewai import Flow
from crewai.flow.flow import listen, start

class dbtChatFlow(Flow):
    @start()
    def interpret_prompt(self):
        user_prompt = self.state["user_input"]
        print(user_prompt)
        interpretation_result = crew.kickoff(inputs = {'request': user_prompt} )
        self.state["interpretation_result"] = interpretation_result
        return interpretation_result

    @listen(lambda state: "interpretation_result" in state)
    def evaluate_interpretation(self):
        interpretation_result = self.state.get("interpretation_result")
        evaluation_result = crew.agents[1].kickoff({"request": interpretation_result})
        self.state["evaluation_result"] = evaluation_result
        return evaluation_result

flow = dbtChatFlow()
#flow.plot()

In [13]:
user_input = 'Give me all the information about the models related with customers'
flow.kickoff(inputs={"user_input": user_input})

Give me all the information about the models related with customers
# Agent: Request Interpreter
## Task: Evaluate the user's request: Give me all the information about the models related with customers and determine the required action: - Adding a field: Identify where the field is currently available (if provided),
  determine how to propagate it through necessary models or transformations,
  and assess the impact on related models and dependencies.
- Modifying an existing model: Identify specific changes needed, evaluate the
  impact on structure, relationships, and downstream dependencies.
- Retrieving specific information: Identify models containing relevant data,
  analyze relationships, and determine queries or transformations needed.

Reflect on the request to generate a concise plan for the approach and provide a clear summary of the required action and its implications.



# Agent: Request Interpreter
## Final Answer: 
RETRIEVE_INFO - The user is requesting all information ab

CrewOutput(raw="To fulfill the user request for all information about models related to customers, follow these high-level actionable steps:\n\n1. **Identify Target Models or Files**: \n   - Locate and list all dbt models that contain customer-related data. This could include models directly named 'customers' as well as any associated models such as 'customer_orders', 'customer_segments', or 'customer_addresses'.\n\n2. **Analyze Dependencies and Relationships**: \n   - Examine upstream and downstream dependencies for each identified model using dbt's lineage graph. Document which models feed into customer-related models and which models derive data from them.\n\n3. **Field Existence Evaluation**: \n   - Check the schema of each customer-related model to ensure that all necessary fields (e.g., customer ID, name, contact information) are present and correctly defined.\n\n4. **Assess Documentation Needs**: \n   - Review existing documentation for the identified models. Identify gaps in do

### CREATE AGENT CHAIN

#### Configure Prerequisites

In [13]:
from langchain_openai import ChatOpenAI

loaded_vectorstore = Chroma(
    collection_name=COLLECTION_NAME,
    persist_directory=CHROMADB_DIRECTORY,
    embedding_function=langchain_openai_embeddings
)

/var/folders/hv/ckh3m6gn1sd45q2qctrqcwzh0000gn/T/ipykernel_1563/2024057643.py:3: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  loaded_vectorstore = Chroma(


In [14]:
_, repo_name = generate_knowledge.extract_owner_and_repo('https://github.com/dbt-labs/jaffle-shop')
dbt_models_df = pd.read_csv('../data/dbt_models_' + repo_name + '.csv')
dbt_project_df = pd.read_csv('../data/dbt_project_' + repo_name + '.csv')
dbt_repo_knowledge_df = create_rag_db.merge_dbt_models_and_project_dfs(dbt_models_df, dbt_project_df)

In [15]:
retriever = EnhancedRetriever(vectorstore = loaded_vectorstore, embedding_function= langchain_openai_embeddings)

query = "give me all the models related with the dbt model orders"
final_context, top_documents = retriever.retrieve(query)

#### Create agents, tasks and flow

In [16]:
# Creating Agents
interpretation_agent = Agent(
  config=agents_config['interpretation_agent'],
)

evaluation_agent = Agent(
  config=agents_config['evaluation_agent'],
)

lineage_agent = Agent(
  config=agents_config['lineage_agent'],
)

plan_agent = Agent(
  config=agents_config['plan_agent'],
)

In [17]:
# Creating Tasks
interpretation_task = Task(
  config=tasks_config['interpretation_task'],
  agent=interpretation_agent
)

evaluation_task = Task(
  config=tasks_config['evaluation_task'],
  agent=evaluation_agent
)

lineage_task = Task(
  config=tasks_config['lineage_task'],
  agent=lineage_agent
)

plan_task = Task(
  config=tasks_config['plan_task'],
  agent=plan_agent
)

In [18]:
interpretation_crew = Crew(agents = [interpretation_agent], tasks = [interpretation_task], verbose = True)
evaluation_crew = Crew(agents = [evaluation_agent], tasks = [evaluation_task], verbose = True)
lineage_crew = Crew(agents = [lineage_agent], tasks = [lineage_task], verbose = True)
plan_crew = Crew(agents = [plan_agent], tasks = [plan_task], verbose = True)

Overriding of current TracerProvider is not allowed
Overriding of current TracerProvider is not allowed
Overriding of current TracerProvider is not allowed
Overriding of current TracerProvider is not allowed


In [21]:
from crewai import Flow
from crewai.flow.flow import listen, start

class dbtChatFlow(Flow):
    @start()
    def interpret_prompt(self):
        request = self.state["request"]
        interpretation = interpretation_crew.kickoff(inputs = {'request': request})
        self.state["interpretation"] = interpretation
        return interpretation

    @listen(interpret_prompt)
    def evaluate_interpretation(self):
        request = self.state["request"]
        interpretation = self.state.get("interpretation")
        evaluation = evaluation_crew.kickoff(inputs = {'request': request, "interpretation": interpretation})
        self.state["evaluation"] = evaluation
        return evaluation
    
    @listen(evaluate_interpretation)
    def retrieve_general_context_for_lineage_calculation(self):
        request = self.state["request"]
        vectorstore = self.state["vectorstore"]
        embedding_function = self.state["embedding_function"]
        retriever = EnhancedRetriever(vectorstore = vectorstore, embedding_function= embedding_function)
        retrieved_context, retrieved_documents = retriever.retrieve(request)
        retrieved_context = "\n".join([doc.page_content for doc in retrieved_documents if hasattr(doc, 'page_content')])
        self.state["retrieved_context"] = retrieved_context
        return retrieved_context

    @listen(retrieve_general_context_for_lineage_calculation)
    def get_lineage(self):
        request = self.state["request"]
        evaluation = self.state.get("evaluation")
        
        retrieved_context = self.state.get("retrieved_context")
        lineage_analysis = lineage_crew.kickoff(inputs = {'request': request, 'evaluation': str(evaluation), 'retrieved_context':retrieved_context})
        self.state["lineage_analysis"] = lineage_analysis
        return lineage_analysis

flow = dbtChatFlow()
flow.plot()

Plot saved as crewai_flow.html


In [22]:
user_input = 'Give me all the information about the models related with customers'
flow.kickoff(inputs={"request": user_input, "vectorstore": loaded_vectorstore, "embedding_function":langchain_openai_embeddings})

# Agent: Request Interpreter
## Task: Evaluate the user's request: Give me all the information about the models related with customers and based on the evaluation, determine the required action: 1) adding a field -> return ADD_COLUMN 2) modifying an existing model -> return MODIFY_MODEL 3) retrieving and returning specific information. -> return RETRIVE_INFO
Reflect on the request and provide a concise plan for the approach: - If the action involves adding a field: Identify where the field is currently available, if provided. - Determine how to propagate the field through the necessary models or transformations to integrate it into the target model. - Consider the impact on related models and dependencies. - If the action involves modifying an existing model: Identify the specific changes required. - Assess how these changes affect the structure, relationships, and downstream dependencies of the model. - If the action involves retrieving or returning information: Identify the models co

Traceback (most recent call last):
  File "/Users/jobandtalent/opt/anaconda3/envs/rag-env/lib/python3.10/site-packages/crewai/task.py", line 535, in interpolate_only
    raise ValueError(
ValueError: Value for key 'evaluation' must be a string, integer, or float, got CrewOutput

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/Users/jobandtalent/opt/anaconda3/envs/rag-env/lib/python3.10/site-packages/crewai/task.py", line 486, in interpolate_inputs
    self.expected_output = self.interpolate_only(
  File "/Users/jobandtalent/opt/anaconda3/envs/rag-env/lib/python3.10/site-packages/crewai/task.py", line 550, in interpolate_only
    raise ValueError(f"Error during string interpolation: {str(e)}") from e
ValueError: Error during string interpolation: Value for key 'evaluation' must be a string, integer, or float, got CrewOutput

The above exception was the direct cause of the following exception:

Traceback (most recent call 

'{\'models\': [{\'name\': \'customers\', \'description\': \'Customer overview data mart, offering key details for each unique customer. One row per customer.\', \'data_tests\': [{\'dbt_utils.expression_is_true\': {\'expression\': \'lifetime_spend_pretax + lifetime_tax_paid = lifetime_spend\'}}], \'columns\': [{\'name\': \'customer_id\', \'description\': \'The unique key of the orders mart.\', \'data_tests\': [\'not_null\', \'unique\']}, {\'name\': \'customer_name\', \'description\': "Customers\' full name."}, {\'name\':\n<MODEL CODE>:\n        .sql Code:\n        WITH customers AS\n  (SELECT *\n   FROM {{ ref(\'stg_customers\') }}),\n     orders AS\n  (SELECT *\n   FROM {{ ref(\'orders\') }}),\n     customer_orders_summary AS\n  (SELECT orders.customer_id,\n          count(DISTINCT orders.order_id) AS count_lifetime_orders,\n          count(DISTINCT orders.order_id) > 1 AS is_repeat_buyer,\n          min(orders.ordered_at) AS first_ordered_at,\n          max(orders.ordered_at) AS last_

#### Example

In [ ]:
from crewai import Flow
from crewai.flow.flow import listen, start, and_, or_, router

class SalesPipeline(Flow):
    
  @start()
  def fetch_leads(self):
    # Pull our leads from the database
    # This is a mock, in a real-world scenario, this is where you would
    # fetch leads from a database
    leads = [
      {
        "lead_data": {
          "name": "João Moura",
          "job_title": "Director of Engineering",
          "company": "Clearbit",
          "email": "joao@clearbit.com",
          "use_case": "Using AI Agent to do better data enrichment."
        },
      },
    ]
    return leads

  @listen(fetch_leads)
  def score_leads(self, leads):
    scores = lead_scoring_crew.kickoff_for_each(leads)
    self.state["score_crews_results"] = scores
    return scores

  @listen(score_leads)
  def store_leads_score(self, scores):
    # Here we would store the scores in the database
    return scores

  @listen(score_leads)
  def filter_leads(self, scores):
    return [score for score in scores if score['lead_score'].score > 70]

  @listen(and_(filter_leads, store_leads_score))
  def log_leads(self, leads):
    print(f"Leads: {leads}")

  @router(filter_leads, paths=["high", "medium", "low"])
  def count_leads(self, scores):
    if len(scores) > 10:
      return 'high'
    elif len(scores) > 5:
      return 'medium'
    else:
      return 'low'

  @listen('high')
  def store_in_salesforce(self, leads):
    return leads

  @listen('medium')
  def send_to_sales_team(self, leads):
    return leads

  @listen('low')
  def write_email(self, leads):
    scored_leads = [lead.to_dict() for lead in leads]
    emails = email_writing_crew.kickoff_for_each(scored_leads)
    return emails

  @listen(write_email)
  def send_email(self, emails):
    # Here we would send the emails to the leads
    return emails